In [1]:
import math
from scipy.stats import rankdata
import pandas as pd

In [2]:
def position_angles(rows, sumdelegates):
    
    Totals = [4, 15, 33, 61, 95] # Threshold for new row. For instance, more than 15 seats means 3 rows
    
    # Maximum radius of spot is 0.5/rows; leave a bit of space.
    radius = 0.4/rows

    # Create list of centre spots
    poslist_24 = []

    for i in range(1, rows):
        """ The relevant math is borrowed from David Richfield (slashme on GitHub)"""
        
        # Each row can contain pi/(2asin(2/(3n+4i-2))) spots, where n is the number of rows and i is the number
        # of the current row. Fill each row proportionally to the "fullness" of the diagram, up to the second-last row.
        
        J = int(float(sumdelegates) / Totals[rows-1] * math.pi/(2*math.asin(2.0/(3.0*rows+4.0*i-2.0))))
        
        # The radius of the ith row in an N-row diagram (Ri) is (3*N+4*i-2)/(4*N)
        
        R = (3.0*rows+4.0*i-2.0)/(4.0*rows)
        
        if J == 1:
            poslist_24.append([math.pi/2.0, 1.75*R, R])
        else:
            for j in range(J):
                # The angle to a spot is n.(pi-2sin(r/Ri))/(Ni-1)+sin(r/Ri) where Ni is the number in the arc
                # x=R.cos(theta) + 1.75
                # y=R.sin(theta)
                
                angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J)-1.0)+math.sin(radius/R)   
                poslist_24.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])

                
    # Now whatever seats are left go into the outside row:
    J_2 = sumdelegates-len(poslist_24)
    R = (7.0*rows-2.0)/(4.0*rows)
    if J_2 == 1:
        poslist_24.append([math.pi/2.0, 1.75*R, R])
    else:
        for j in range(J_2):
            angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J_2)-1.0)+math.sin(radius/R)
            poslist_24.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])
    poslist_24.sort(reverse = True)
    poslist_data = pd.DataFrame(data = poslist_24, columns = ['angle', 'x', 'y'])
    
    return poslist_data

In [3]:
dataframe = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\seat_allocation.xlsx')

poslist_final = pd.DataFrame(columns = ['angle', 'x', 'y'])

dataframe['rows'] = 3
dataframe.loc[dataframe['council_seats'] >= 33,  'rows'] = 4
dataframe.loc[dataframe['council_seats'] >= 61,  'rows'] = 5

for i in range(len(dataframe.city.unique())):
    
    city_now = dataframe.city.unique()[i]
    dataframe_1 = dataframe[(dataframe.city == city_now)].reset_index()
    row = dataframe_1.rows[0]
    no_seats = dataframe_1.council_seats[0]
    
    poslist_final = pd.concat([poslist_final, position_angles(row, no_seats)])

poslist_final

,angle,x,y
0,3.080093,0.128072,0.099874
1,3.068929,0.378628,0.099824
2,3.052821,0.629430,0.099737
3,3.027556,0.880683,0.099566
4,2.828544,0.203977,0.500437
...,...,...,...
45,0.250162,3.324418,0.402286
46,0.114037,2.619317,0.099566
47,0.088772,2.870570,0.099737
48,0.072663,3.121372,0.099824


In [4]:
data_graph = dataframe.join(poslist_final.reset_index())

In [5]:
data_graph['color'] = '#999999'

data_graph.loc[data_graph['party'] == 'BP', 'color'] = '#98F5FF'
data_graph.loc[data_graph['party'] == 'Partei', 'color'] = '#B5152B'
data_graph.loc[data_graph['party'] == 'AfD', 'color'] = '#87CEFF'
data_graph.loc[data_graph['party'] == 'ödp', 'color'] = '#698B22'
data_graph.loc[data_graph['party'] == 'FW', 'color'] = '#FFA500'
data_graph.loc[data_graph['party'] == 'Linke', 'color'] = '#A020F0'
data_graph.loc[data_graph['party'] == 'BIA', 'color'] = '#8B4513'
data_graph.loc[(data_graph['party'] != 'ödp') & (data_graph['party'] != 'Partei') & (data_graph['left_index'] == 4)
               , 'color'] = '#EEE5DE'
data_graph.loc[(data_graph['party'] != 'Linke') & (data_graph['left_index'] == 1), 'color'] = '#DDA0DD'
data_graph.loc[data_graph['party'] == 'FDP', 'color'] = '#FFFF00'
data_graph.loc[data_graph['party'] == 'Grüne', 'color'] = '#00AA00'
data_graph.loc[data_graph['party'] == 'SPD', 'color'] = '#FF0000'
data_graph.loc[data_graph['party'] == 'CSU', 'color'] = '#000000'



In [6]:
data_graph_1 = data_graph[:896].reset_index()
data_graph_2 = data_graph[896:].reset_index()

In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.offline as pyo
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import tools
import chart_studio

chart_studio.tools.set_credentials_file(username='marius92', api_key='4naxu3XiGZTGniWO02z1')

In [92]:
# Create individual graphs

fig = go.Figure()

list_parties= data_graph_2.party.unique()
list_cities = data_graph_2.city.unique()

for j in range(len(list_cities)):
    
    city_now = list_cities[j]
    data_trace = data_graph_2[data_graph_2['city'].str.match(city_now)]
    list_parties = data_trace.party.unique()
    
    if j == 0:
        for i in range(len(list_parties)):
            
            party_now = list_parties[i]
            
            fig.add_trace(go.Scatter(
                x= data_trace[data_trace['party'].str.match(party_now)]['x'], 
                y= data_trace[data_trace['party'].str.match(party_now)]['y'],
                mode='markers',
                name = party_now,
                marker = dict(
                    size = 24,
                    color = data_trace[data_trace['party'].str.match(list_parties[i])]['color']),
                hoverinfo = 'text',
                hovertext = party_now,
                visible = False, # Change to forecasted politician/or just number of seat for party ?
            )
            )
    
    
    if j != 0:       
    # Generate a scatter of a certain color for every party
        for i in range(len(list_parties)):
            party_now = list_parties[i]
    
            fig.add_trace(go.Scatter(
                x= data_trace[data_trace['party'].str.match(party_now)]['x'], 
                y= data_trace[data_trace['party'].str.match(party_now)]['y'],
                mode='markers',
                name = party_now,
                marker = dict(
                    size = 24,
                    color = data_trace[data_trace['party'].str.match(list_parties[i])]['color']),
                hoverinfo = 'text',
                hovertext = party_now,
                visible = False, # Change to forecasted politician/or just number of seat for party ?
            )
            )
        
    x_annot = (data_trace['x'].iloc[-1] / 2) + data_trace['x'].iloc[0] # x center alignment for text
    y_max = data_trace['y'].max()
        
    fig.layout.update(
        legend = dict(orientation='v', # Put in a legend
                        font = dict(family = "Old Standard TT",
                                    size = 24,
                                    color = 'black',
                                    ),
                        x = -0.5019,
                        y = data_graph_2['y'].iloc[0] ,
                ),
    )
        
    fig.layout.update(
    xaxis = dict(showgrid=False, zeroline=False, visible = False), # Hide anything related to the x-axis
    yaxis = dict(showgrid=False, zeroline=False, visible = False), # Hide anything related to the y-axis
    plot_bgcolor = 'white',
    title = dict(font = dict(family = 'Old Standard TT',
                             size = 32,
                             color = 'black',
                        ),
                 x = 0.64,
            ),
    )

In [93]:
button_data = data_graph_2.groupby(['city', 'party'], as_index = False).mean()
data_button.to_excel(r'C:\Users\mariu\Documents\Project Local Elections\weird_3.xlsx')

In [94]:
button_data = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\weird_3.xlsx')

In [95]:
button_data_2 = pd.DataFrame()

for i in range(len(button_data.city.unique())):
    
    name = list_cities[i]
    button_data[name] = False
    button_data.loc[button_data['city'] == name, name] = True
    

In [98]:
buttons=[]

for i in range(len(button_data.city.unique())):
    
    name = button_data.city.unique()[i]
    buttons.append(dict(method='update',
                        label= '<b>' +  button_data.city.unique()[i] + '</b>',
                        args=[{'visible': button_data[name].values,
                               'title': " <b>" + button_data.city.unique()[i] + "</b>"}, ],
                        )
                  )


In [99]:
fig.layout.update(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=buttons,
            x = data_graph['x'].iloc[0] - 0.4,
            y = 1.24,
            bgcolor = '#efefef',
            font = dict(family = 'Old Standard TT',
                        size = 18,
                       ),
            borderwidth = 1.2,
        )
    ])

fig['layout'].update(width = 1000, height = 500)
filename = 'Parliament_seats_2'
py.iplot(fig, filename = filename)